# Metadata

**L1 Taxonomy** - Computing Paradigms

**L2 Taxonomy** - Procedural Programming

**Subtopic** - Procedural macro that generates highly optimized DP functions based on annotated recurrence definitions
  

**Programming Language** - Rust  
**Target Model** - o1  



# Prompt


Implement a highly optimized Rust solution for the Traveling Salesman Problem on a small complete graph. Use bitmask dynamic programming accelerated with SIMD intrinsics. Compute the minimum-cost Hamiltonian cycle that visits every city exactly once and returns to the start. Leverage Rust's `std::arch` module for vectorized distance updates, falling back to a scalar DP loop if SIMD isn't available at runtime.

**Input Format and Constraints**

* Read from standard input.
* First line: integer `N`, where `0 ≤ N ≤ 16`.
* If `N = 0`: print `0` and exit.
* Next `N` lines: each contains `N` non-negative integers (each <= 1_000_000), space-separated, giving the distance matrix `dist[i][j]`.
* The graph is complete and symmetric (`dist[i][j] == dist[j][i]`), with `dist[i][i] == 0`.

**Expected Output Format**

* A single integer: the length of the shortest Hamiltonian cycle.

**Examples**

```
Input
4
0 29 20 21
29 0 15 17
20 15 0 28
21 17 28 0

Output
80
```

```
Input
3
0 10 15
10 0 20
15 20 0

Output
45
```



# Requirements

**Explicit and Implicit Points**

* Implement the DP recurrence:
  `dp[mask][i] = min_{j ∈ mask \ {i}} (dp[mask \ {i}][j] + dist[j][i])`.
* Use `unsafe` only for SIMD intrinsics from `std::arch::x86_64` or `std::arch::aarch64`.
* Handle `N = 0` and `N = 1` by printing `0`.
* Document CPU assumptions: require AVX2 (or NEON) at runtime, otherwise fall back to scalar.

**Solution Expectations**

* Time complexity: O(N · 2^N / lane_width) in the innermost loop.
* Memory complexity: O(2^N · N).
* Must compile on Rust stable 1.70+ with no external crates.
* No panics or integer overflows; use `u32` with `saturating_add` (or `u64`).
* Annotate where SIMD optimizations are applied.

Deliverables

The solution will consist of three files only:
- Use
- lib.rs — contains the TSP logic and SIMD acceleration
- main.rs — reads input, calls solver, writes output
- build.rs Not really needed.

**Function Signature**

```rust
fn solve_tsp<R: std::io::BufRead, W: std::io::Write>(
    input: &mut R,
    output: &mut W
) -> std::io::Result<()>;
```

**Structures**

```rust
pub struct DpSolver {
    pub n: usize,
    pub dist: Vec<Vec<u32>>,
    pub dp: Vec<u32>,
}

impl DpSolver {
    pub fn new(n: usize, dist: Vec<Vec<u32>>) -> Self;
    /// Unsafe SIMD‑accelerated DP; falls back automatically.
    pub unsafe fn compute_simd(&mut self, full_mask: usize) -> u32;
}
```

**Edge-Case Behavior**

* `N = 0`: print `0`.
* `N = 1`: print `0`.
* Malformed input (wrong row length): return an `Err`.

**Blind-Spot Edge Cases**

* Distances near `u32::MAX` so that additions risk overflow.
* `N` not a multiple of SIMD lane width (e.g. lane = 8, `N = 6`): correctly handle tail elements.
* Running on a CPU without the chosen SIMD feature: detect at runtime and fall back to scalar.

**Constraints**

* No external crates beyond the standard library.
* No recursive DP that can overflow the stack at `N = 16`.
* No macros that conceal algorithmic complexity.



# build

```rust
// build.rs   generic stub, safe for every task
fn main() {
    // Re‑run only if these files change (keeps incremental builds quick)
    println!("cargo:rerun-if-changed=build.rs");
    println!("cargo:rerun-if-changed=Cargo.toml");

    // Emit a cfg flag that downstream code may (or may not) use
    println!("cargo:rustc-cfg=build_script_generated");
}

```

# lib

```rust
// src/lib.rs

use std::io::{self, BufRead, Write};

#[cfg(target_arch = "x86_64")]
use std::arch::x86_64::{
    __m256i, _mm256_add_epi32, _mm256_loadu_si256, _mm256_min_epu32, _mm256_set1_epi32,
    _mm256_storeu_si256,
};

/// Solver for the bitmask‐DP Traveling Salesman Problem.
pub struct DpSolver {
    pub n: usize,
    pub dist: Vec<Vec<u32>>,
    pub dp: Vec<u32>,
}

impl DpSolver {
    /// Initialize a new solver for `n` cities with the given distance matrix.
    pub fn new(n: usize, dist: Vec<Vec<u32>>) -> Self {
        let size = (1 << n) * n;
        let mut dp = vec![u32::MAX; size];
        if n > 0 {
            dp[(1 << 0) * n + 0] = 0;
        }
        DpSolver { n, dist, dp }
    }

    /// Compute the shortest Hamiltonian cycle length.
    ///
    /// Uses AVX2 SIMD if detected at runtime, otherwise falls back to scalar.
    /// Returns 0 immediately for n ≤ 1.
    pub fn compute(&mut self) -> u32 {
        if self.n <= 1 {
            return 0;
        }
        let full_mask = (1 << self.n) - 1;
        #[cfg(target_arch = "x86_64")]
        {
            if is_x86_feature_detected!("avx2") {
                // SAFETY: AVX2 support was checked
                return unsafe { self.compute_simd(full_mask) };
            }
        }
        self.compute_scalar(full_mask)
    }

    /// Scalar fallback implementation.
    fn compute_scalar(&mut self, full: usize) -> u32 {
        let n = self.n;
        for mask in 1..=full {
            for i in 0..n {
                if mask & (1 << i) == 0 { continue; }
                let prev = mask ^ (1 << i);
                if prev == 0 {         // keep the seed dp[1*n + 0] = 0
                    continue;
                }
                let base_prev = prev * n;
                let idx = mask * n + i;
                let mut best = u32::MAX;
                for j in 0..n {
                    if prev & (1 << j) != 0 {
                        let cost = self.dp[base_prev + j].saturating_add(self.dist[j][i]);
                        if cost < best { best = cost; }
                    }
                }
                self.dp[idx] = best;
            }
        }
        // close cycle
        let mut result = u32::MAX;
        for i in 0..n {
            let cost = self
                .dp[full * n + i]
                .saturating_add(self.dist[i][0]);
            if cost < result {
                result = cost;
            }
        }
        result
    }

    /// Unsafe SIMD‐accelerated implementation (AVX2).
    #[target_feature(enable = "avx2")]
    pub unsafe fn compute_simd(&mut self, full_mask: usize) -> u32 {
        let n = self.n;
        let lane = 8;
        let chunks = n / lane;
        for mask in 1..=full_mask {
            for i in 0..n {
                if mask & (1 << i) == 0 { continue; }
                let prev = mask ^ (1 << i);
                if prev == 0 {                 continue;           }
                let base = mask * n + i;
                let base_prev = prev * n;

                let mut best_vec: __m256i = _mm256_set1_epi32(-1);
                for c in 0..chunks {
                    let j0 = c * lane;
                    let dp_ptr = self.dp.as_ptr().add(base_prev + j0) as *const __m256i;
                    let dp_vec = _mm256_loadu_si256(dp_ptr);

                    let mut ds = [0u32; 8];
                    for k in 0..lane {
                        ds[k] = self.dist[j0 + k][i];
                    }
                    let dist_vec = _mm256_loadu_si256(ds.as_ptr() as *const __m256i);

                    let sum = _mm256_add_epi32(dp_vec, dist_vec);
                    best_vec = _mm256_min_epu32(best_vec, sum);
                }

                let mut tmp = [0u32; 8];
                _mm256_storeu_si256(tmp.as_mut_ptr() as *mut __m256i, best_vec);
                let mut best = tmp.iter().cloned().min().unwrap_or(u32::MAX);

                for j in (chunks * lane)..n {
                    if prev & (1 << j) != 0 {
                        let cost = self.dp[base_prev + j].saturating_add(self.dist[j][i]);
                        if cost < best { best = cost; }
                    }
                }

                self.dp[base] = best;
            }
        }
        // close cycle
        let mut result = u32::MAX;
        for i in 0..n {
            let cost = self
                .dp[full_mask * n + i]
                .saturating_add(self.dist[i][0]);
            if cost < result {
                result = cost;
            }
        }
        result
    }
}

/// Parse input, validate, run the solver, and write output.
pub fn solve_tsp<R: BufRead, W: Write>(
    input: &mut R,
    output: &mut W,
) -> io::Result<()> {
    let mut buf = String::new();
    input.read_line(&mut buf)?;
    let n: usize = buf.trim().parse().map_err(|_| {
        io::Error::new(io::ErrorKind::InvalidData, "Invalid N")
    })?;

    if n == 0 {
        writeln!(output, "0")?;
        return Ok(());
    }

    let mut dist = Vec::with_capacity(n);
    for line_idx in 0..n {
        buf.clear();
        input.read_line(&mut buf)?;
        let row: Vec<u32> = buf
            .split_whitespace()
            .map(|s| s.parse().unwrap_or(u32::MAX))
            .collect();
        if row.len() != n {
            return Err(io::Error::new(
                io::ErrorKind::InvalidData,
                format!("Line {}: expected {} values, got {}", line_idx + 1, n, row.len()),
            ));
        }
        dist.push(row);
    }

    let mut solver = DpSolver::new(n, dist);
    let ans = solver.compute();
    writeln!(output, "{}", ans)?;
    Ok(())
}

```

# main

```rust
// src/main.rs

use std::io::{self, BufRead, Write};
use task_ws::solve_tsp;

fn main() -> io::Result<()> {
    let stdin = io::stdin();
    let stdout = io::stdout();
    solve_tsp(&mut stdin.lock(), &mut stdout.lock())
}
```

# test


```rust

use std::io::Cursor;
use task_ws::solve_tsp;            

/// Helper: run the solver and capture its single‑line output.
fn run_ok(input: &str) -> String {
    let mut rdr = Cursor::new(input);
    let mut out = Vec::<u8>::new();
    solve_tsp(&mut rdr, &mut out).unwrap();
    String::from_utf8(out).unwrap().trim().to_string()
}

/// Helper: assert that the solver returns an error.
fn run_err(input: &str) {
    let mut rdr = Cursor::new(input);
    let mut out = Vec::<u8>::new();
    assert!(solve_tsp(&mut rdr, &mut out).is_err());
}

/* ---------- malformed‑input checks ---------- */

#[test] fn invalid_n()                 { run_err("foo\n"); }

#[test] fn bad_row_count()             { run_err(r#"3
0 1 2
3 4 5
"#); }

#[test] fn bad_row_too_short()         { run_err(r#"2
0
0 0
"#); }

#[test] fn bad_row_too_long()          { run_err(r#"2
0 1 2
0 0
"#); }

/* ---------- trivial sizes ---------- */

#[test] fn n_zero()                   { assert_eq!(run_ok("0\n"), "0"); }

#[test] fn n_one()                    { assert_eq!(run_ok("1\n0\n"), "0"); }

/* ---------- prompt examples ---------- */

#[test]
fn example_four_city() {
    let input = "4\n\
                 0 29 20 21\n\
                 29 0 15 17\n\
                 20 15 0 28\n\
                 21 17 28 0\n";
    assert_eq!(run_ok(input), "73");
}

#[test]
fn example_three_city() {
    let input = "3\n\
                 0 10 15\n\
                 10 0 20\n\
                 15 20 0\n";
    assert_eq!(run_ok(input), "45");
}
/* ---------- edge cases & blind spots ---------- */

#[test]
fn overflow_saturates() {
    let half = std::u32::MAX / 2;
    let expect = (half * 2).to_string();              // 4 294 967 294
    let inp = format!("2\n0 {}\n{} 0\n", half, half);
    assert_eq!(run_ok(&inp), expect);
}


#[test]
fn simd_tail_handling() {
    // N = 10, not a multiple of 8‑lane AVX2
    let mut inp = String::from("10\n");
    for _ in 0..10 { inp.push_str(&"0 ".repeat(10)); inp.push('\n'); }
    assert_eq!(run_ok(&inp), "0");
}

#[test]
fn simd_exact_lane() {
    // N = 8: exactly one AVX2 vector, all zeros
    let mut inp = String::from("8\n");
    for _ in 0..8 { inp.push_str(&"0 ".repeat(8)); inp.push('\n'); }
    assert_eq!(run_ok(&inp), "0");
}

#[test]
fn all_zero_n16() {
    let mut inp = String::from("16\n");
    for _ in 0..16 { inp.push_str(&"0 ".repeat(16)); inp.push('\n'); }
    assert_eq!(run_ok(&inp), "0");
}

```
